In [1]:
# Import necessary libraries
import os
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

# --- 1. CONNECT TO THE DATABASE ---

# Load environment variables from .env file
# Make sure your .env file is in the root of your project directory
load_dotenv()

# Get database credentials from environment variables
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

# Create the database connection string
db_url = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Create a SQLAlchemy engine
engine = create_engine(db_url)
print("Successfully connected to the PostgreSQL database.")


# --- 2. DEFINE THE SQL QUERY ---

# This query joins all our tables to create a single analytical dataset.
# It calculates the number of news articles for each stock on each day.
query = text("""
    SELECT
        d.full_date,
        t.ticker_symbol,
        p.open_price,
        p.high_price,
        p.low_price,
        p.close_price,
        p.adj_close_price,
        p.volume,
        COALESCE(n.news_count, 0) AS news_count -- If a day has no news, count is 0
    FROM 
        fact_stock_prices AS p
    JOIN 
        dim_tickers AS t ON p.ticker_id = t.ticker_id
    JOIN 
        dim_dates AS d ON p.date_id = d.date_id
    LEFT JOIN (
        -- Subquery to count news articles per ticker per day
        SELECT
            ticker_id,
            date_id,
            COUNT(article_id) AS news_count
        FROM 
            fact_news_articles
        GROUP BY 
            ticker_id, date_id
    ) AS n ON p.ticker_id = n.ticker_id AND p.date_id = n.date_id
    ORDER BY 
        t.ticker_symbol, d.full_date;
""")


# --- 3. EXECUTE THE QUERY AND LOAD DATA INTO PANDAS ---

with engine.connect() as connection:
    df = pd.read_sql(query, connection)

# Display the first few rows and info to verify
print("\nData successfully fetched from the database.")
print("DataFrame Info:")
df.info()

print("\nFirst 5 rows of the combined data:")
df.head()

Successfully connected to the PostgreSQL database.

Data successfully fetched from the database.
DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315 entries, 0 to 314
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   full_date        315 non-null    object 
 1   ticker_symbol    315 non-null    object 
 2   open_price       315 non-null    float64
 3   high_price       315 non-null    float64
 4   low_price        315 non-null    float64
 5   close_price      315 non-null    float64
 6   adj_close_price  315 non-null    float64
 7   volume           315 non-null    int64  
 8   news_count       315 non-null    int64  
dtypes: float64(5), int64(2), object(2)
memory usage: 22.3+ KB

First 5 rows of the combined data:


,full_date,ticker_symbol,open_price,high_price,low_price,close_price,adj_close_price,volume,news_count
0,2025-06-02,AAPL,200.279999,202.130005,200.119995,201.699997,201.471344,35423300,0
1,2025-06-03,AAPL,201.350006,203.770004,200.960007,203.270004,203.039566,46381600,0
2,2025-06-04,AAPL,202.910004,206.240005,202.100006,202.820007,202.590088,43604000,0
3,2025-06-05,AAPL,203.500000,204.750000,200.149994,200.630005,200.402573,55126100,0
4,2025-06-06,AAPL,203.000000,205.699997,202.050003,203.919998,203.688828,46607700,0
